# Setup Environment
This notebook sets up the rest of the demonstration used to create and deploy a Azure Machine Learning Model (Classification).

**Data Used:**
The notebook uses the following Credit card Payment Default data: https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients

### Check AzureML SDK
Ensure we have a properly installed SDK

In [1]:
import azureml.core
print(azureml.core.VERSION)

1.2.0


### Get Subscription details
If you don't know what it is, you can get it from the Azure Command Line Interface (CLI). 
To install the Azure CLI see https://docs.microsoft.com/en-us/cli/azure/install-azure-cli

In [2]:
result=!az account show -s "Microsoft Azure Internal Consumption" --query id -o tsv
tenant_id=result[0]
print(f"Found Tenant ID: {tenant_id}")

result=!az account show -s "Microsoft Azure Internal Consumption" --query id -o tsv
subscription_id=result[0]
print(f"Found Subscription ID: {subscription_id}")

Found Tenant ID: b9f1c816-0637-419c-b0f6-a8392690c7aa
Found Subscription ID: b9f1c816-0637-419c-b0f6-a8392690c7aa


In [8]:
workspace_name     = "paydmlws91"
resource_group     = "paydmlrg91"
dc_region_location = "westeurope"
sku                = "enterprise"     # basic or enterprise
friendly_name      = "My Workspace"   # Displayed in the Workspace

### Create a Workspace
For more information see: https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/manage-azureml-service/authentication-in-azureml/authentication-in-azureml.ipynb

In [3]:
from azureml.core.authentication import AzureCliAuthentication
from azureml.core.workspace import Workspace
import pandas as pd

cli_auth = AzureCliAuthentication()

ws = Workspace.create(name=workspace_name,
                      subscription_id=subscription_id,
                      resource_group=resource_group,
                      location=dc_region_location,
                      auth=cli_auth, 
                      sku=sku)

ws.write_config()      # Save Workspace config file

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

Deploying AppInsights with name paydmlwsinsights5f59ee04.
Deployed AppInsights with name paydmlwsinsights5f59ee04. Took 10.9 seconds.
Deploying KeyVault with name paydmlwskeyvault97552fb6.
Deploying StorageAccount with name paydmlwsstorage944bdd9b3.
Deployed KeyVault with name paydmlwskeyvault97552fb6. Took 24.99 seconds.
Deployed StorageAccount with name paydmlwsstorage944bdd9b3. Took 34.69 seconds.
Deploying Workspace with name paydmlws91.
Deployed Workspace with name paydmlws91. Took 33.56 seconds.


,
SDK version,1.2.0
Subscription ID,b9f1c816-0637-419c-b0f6-a8392690c7aa
Workspace,paydmlws91
Resource Group,paydmlrg91
Location,westeurope


In [5]:
from azureml.core.authentication import AzureCliAuthentication
from azureml.core.workspace import Workspace
import pandas as pd

cli_auth = AzureCliAuthentication()
ws = Workspace.from_config(auth=cli_auth)
ws.get_details()

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

,
SDK version,1.2.0
Subscription ID,b9f1c816-0637-419c-b0f6-a8392690c7aa
Workspace,paydmlws91
Resource Group,paydmlrg91
Location,westeurope


### Create Blob Storage for our Data 
For more information see: https://docs.microsoft.com/en-us/samples/azure-samples/storage-python-manage/storage-python-manage/#create-account

In [6]:
from azure.mgmt.storage import StorageManagementClient

storage_client = StorageManagementClient(cli_auth, subscription_id)

In [9]:
import time
from azure.mgmt.storage.models import StorageAccountCreateParameters
from azure.mgmt.storage.models import Sku,Kind

storage_client = StorageManagementClient(cli_auth, subscription_id)

storage_prefix='paydef'
storage_account_name = storage_prefix + time.strftime('%Y%m%d%H%M%S')

print(f'Creating : {storage_account_name}')

storage_async_operation = storage_client.storage_accounts.create(
    resource_group,
    storage_account_name,
    StorageAccountCreateParameters(
        sku=Sku(name='Standard_LRS'),
        kind=Kind.storage,
        location=dc_region_location
    )
)
storage_account = storage_async_operation.result()
print('Done')

Creating : paydef20200402210911
Done


### Create a Container in the Storage Account

In [10]:
from azure.storage.blob import BlobServiceClient
from azure.core.exceptions import ResourceExistsError

container_name='payments'

keys = storage_client.storage_accounts.list_keys(resource_group_name=resource_group,
                                                 account_name=storage_account_name)
account_key=keys.keys[0].value

blob_client = BlobServiceClient(account_url=storage_account.primary_endpoints.blob, credential=account_key)

try:
    new_container = blob_client.create_container(container_name)
    properties = new_container.get_container_properties()
    print(f"Created Container: {container_name}")
except ResourceExistsError:
    print("Container already exists.")

Created Container: payments


### Register Blob Storage with the Azure ML Workspace

In [11]:
from azureml.core.datastore import Datastore
from azureml.data.data_reference import DataReference
from msrest.exceptions import HttpOperationError

blobstore_name=container_name + '_data'  # friendly name

try:
    blob_datastore = Datastore.get(ws, blobstore_name)
    print(f"Found ML Blob Datastore with name: {blobstore_name}")
except HttpOperationError:
    blob_datastore = Datastore.register_azure_blob_container(
       workspace=ws,
       datastore_name=blobstore_name,
       account_name=storage_account_name, # Storage account name
       container_name=container_name,     # Name of Azure blob container
       account_key=account_key)           # Storage account key
    print(f"Registered ML blob datastore with name: {blobstore_name}")

Registered ML blob datastore with name: payments_data


### Use the Datastore to Upload the local files

In [14]:
files_2upload=['data/ml_scoring_input/UCI_Credit_Card_INPUT_1.csv',
               'data/ml_scoring_input/UCI_Credit_Card_INPUT_2.csv',
               'data/ml_scoring_input/UCI_Credit_Card_INPUT_3.csv',
               'data/ml_training/UCI_Credit_Card.csv']

dref = blob_datastore.upload_files(files_2upload,
                                   overwrite=False,
                                   show_progress=True) 
print("Done.")

Uploading an estimated of 4 files
Uploading data/ml_scoring_input/UCI_Credit_Card_INPUT_1.csv
Uploading data/ml_scoring_input/UCI_Credit_Card_INPUT_2.csv
Uploading data/ml_scoring_input/UCI_Credit_Card_INPUT_3.csv
Uploading data/ml_training/UCI_Credit_Card.csv
Uploaded data/ml_scoring_input/UCI_Credit_Card_INPUT_1.csv, 1 files out of an estimated total of 4
Uploaded data/ml_scoring_input/UCI_Credit_Card_INPUT_2.csv, 2 files out of an estimated total of 4
Uploaded data/ml_scoring_input/UCI_Credit_Card_INPUT_3.csv, 3 files out of an estimated total of 4
Uploaded data/ml_training/UCI_Credit_Card.csv, 4 files out of an estimated total of 4
Uploaded 4 files
Done.


### Register Dataset with Azure ML
The Dataset makes it easier to:
1. Access the data for training purposes later.
2. It allows users to share datasets
3. Link the Dataset to models for reference purposes

For more info on Datasets see: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.dataset(class)

In [17]:
from azureml.core import Dataset

dataset_name='payment_data'
input_data_path='ml_training/*.csv'

dataset = Dataset.Tabular.from_delimited_files(path = [(blob_datastore, input_data_path)])

dataset = dataset.register(workspace=ws,
                           name=dataset_name,
                           description='Default Payments Data')

### Save the storage account details to Environment Vars
This is not actually used going forward, but it is to illustrate a concept. We retrieve this name in the next notebook.

In [20]:
keyvault = ws.get_default_keyvault()

keyvault.set_secret(name="DATASTORENAME", value = blobstore_name)
keyvault.set_secret(name="DATASETNAME", value = dataset_name)
print('Saved Storage Account Details to the Workspace KeyVault ')

Saved Storage Account Details to the Workspace KeyVault 
